In [1]:
# Importar librerias
import pandas as pd
import numpy as np

In [2]:
# Ruta del archivo en el entorno de Colab
file_path = 'serie_cgi_10_24.xls'

# Leer las pestañas específicas y crear DataFrames
df_vab_pb = pd.read_excel(file_path, sheet_name='VAB_pb')
df_puestos = pd.read_excel(file_path, sheet_name='Puestos')
df_horas = pd.read_excel(file_path, sheet_name='Horas')

In [3]:
# Filas a conservar (de índice 10 a 29 porque Python usa índices base 0)
rows_to_keep = range(9, 29)

# Nuevos nombres de las columnas
new_columns = [
    'COD', 'Sector', '2016Q1', '2016Q2', '2016Q3', '2016Q4', '2016TOT',
    '2017Q1', '2017Q2', '2017Q3', '2017Q4', '2017TOT', '2018Q1', '2018Q2', '2018Q3', '2018Q4', '2018TOT',
    '2019Q1', '2019Q2', '2019Q3', '2019Q4', '2019TOT', '2020Q1', '2020Q2', '2020Q3', '2020Q4', '2020TOT',
    '2021Q1', '2021Q2', '2021Q3', '2021Q4', '2021TOT', '2022Q1', '2022Q2', '2022Q3', '2022Q4', '2022TOT',
    '2023Q1', '2023Q2', '2023Q3', '2023Q4', '2023TOT', '201624Q1', '2024Q2'
]

# Índices de las columnas a eliminar
columns_to_drop = [7, 13, 19, 25, 31, 37, 43, 49]

# Función para procesar cada DataFrame
def process_dataframe(df):
    # Eliminar columnas específicas por índice
    df = df.drop(df.columns[columns_to_drop], axis=1)
    # Conservar solo las filas especificadas
    df = df.iloc[rows_to_keep]
    # Ajustar el número de columnas antes de renombrar
    df = df.iloc[:, :len(new_columns)]  # Seleccionar hasta el número esperado de columnas
    # Renombrar las columnas
    df.columns = new_columns
    return df

# Aplicar la función a cada DataFrame
df_vab_pb = process_dataframe(df_vab_pb)
df_puestos = process_dataframe(df_puestos)
df_horas = process_dataframe(df_horas)

In [4]:
# Función para dividir dos DataFrames, excluyendo las columnas no numéricas (COD y Sector)
def divide_dataframes(df_numerator, df_denominator):
    # Separar las columnas no numéricas (COD y Sector)
    cod_sector = df_numerator.iloc[:, :2]  # Columnas 0 y 1
    # Seleccionar solo las columnas numéricas
    df_numerator_numeric = df_numerator.iloc[:, 2:]
    df_denominator_numeric = df_denominator.iloc[:, 2:]
    # Realizar la división
    result_numeric = df_numerator_numeric.div(df_denominator_numeric)
    # Unir las columnas no numéricas con el resultado de la división
    result = pd.concat([cod_sector, result_numeric], axis=1)
    # Reemplazar infinitos con NaN (por si hay divisiones por cero)
    result.replace([np.inf, -np.inf], np.nan, inplace=True)
    return result

# Crear el DataFrame "productividad_puestos"
productividad_puestos = divide_dataframes(df_vab_pb, df_puestos)

# Crear el DataFrame "productividad_horas"
productividad_horas = divide_dataframes(df_vab_pb, df_horas)

<ipython-input-4-350d43fdb2b1>:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result.replace([np.inf, -np.inf], np.nan, inplace=True)
<ipython-input-4-350d43fdb2b1>:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result.replace([np.inf, -np.inf], np.nan, inplace=True)


In [5]:
# Clasificar los sectores en "Baja", "Media" y "Alta"
sectores_baja = ['A', 'B', 'C']
sectores_media = ['D', 'E', 'F', 'I', 'M', 'MM', 'MMM', 'N', 'NN', 'NNN']
sectores_alta = ['G', 'H', 'J', 'K', 'L', 'O', 'P']

# Función para clasificar los sectores
def clasificar_sector(cod):
    if pd.isna(cod):  # Manejar casos donde el código sea NaN
        return 'Desconocido'
    elif cod in sectores_baja:
        return 'Baja'
    elif cod in sectores_media:
        return 'Media'
    elif cod in sectores_alta:
        return 'Alta'
    else:
        return 'Desconocido'

# Aplicar la clasificación a los DataFrames
df_vab_pb['Clasificacion'] = df_vab_pb['COD'].apply(clasificar_sector)
df_horas['Clasificacion'] = df_horas['COD'].apply(clasificar_sector)

# Función para ajustar las horas trabajadas según la clasificación
def ajustar_horas(df):
    # Crear una copia del DataFrame para no modificar el original
    df_ajustado = df.copy()

    # Convertir las columnas numéricas a tipo float, excluyendo 'COD', 'Sector' y 'Clasificacion'
    columnas_numericas = df_ajustado.columns[2:-1]  # Asegurar el rango correcto
    df_ajustado[columnas_numericas] = df_ajustado[columnas_numericas].apply(pd.to_numeric, errors='coerce')

    # Función para aplicar el ajuste a una fila
    def ajustar_fila(row):
        clasificacion = row['Clasificacion']
        # Convertir horas a minutos
        row[columnas_numericas] = row[columnas_numericas] * 60

        # Aplicar el factor de ajuste en minutos según la clasificación
        if clasificacion == 'Baja':
            factor_ajuste = 1  # 1 minuto cada 18 minutos
        elif clasificacion == 'Media':
            factor_ajuste = 2  # 2 minutos cada 18 minutos
        elif clasificacion == 'Alta':
            factor_ajuste = 3  # 3 minutos cada 18 minutos
        else:
            factor_ajuste = 0

        # Ajustar las columnas numéricas
        row[columnas_numericas] = row[columnas_numericas] * (1 + factor_ajuste / 18)

        # Convertir de nuevo a horas
        row[columnas_numericas] = row[columnas_numericas] / 60
        return row

    # Aplicar la función fila por fila
    df_ajustado = df_ajustado.apply(ajustar_fila, axis=1)
    return df_ajustado

# Ajustar las horas trabajadas en df_horas
df_horas_ajustado = ajustar_horas(df_horas)

# Función para dividir dos DataFrames, excluyendo las columnas no numéricas (COD y Sector)
def divide_dataframes(df_numerator, df_denominator):
    # Separar las columnas no numéricas (COD y Sector)
    cod_sector = df_numerator.iloc[:, :2]  # Columnas 0 y 1
    # Seleccionar solo las columnas numéricas
    columnas_numericas = df_numerator.columns[2:-1]  # Excluir 'Clasificacion' y no numéricas
    df_numerator_numeric = df_numerator[columnas_numericas]
    df_denominator_numeric = df_denominator[columnas_numericas]

    # Realizar la división (manejar posibles divisiones por cero)
    result_numeric = df_numerator_numeric.div(df_denominator_numeric.replace(0, np.nan))

    # Unir las columnas no numéricas con el resultado de la división
    result = pd.concat([cod_sector, result_numeric], axis=1)
    # Reemplazar infinitos con NaN (por si hay divisiones por cero)
    result.replace([np.inf, -np.inf], np.nan, inplace=True)
    return result

# Crear el DataFrame "productividad_horas_distraccion"
productividad_horas_distraccion = divide_dataframes(df_vab_pb, df_horas_ajustado)

productividad_horas_distraccion

<ipython-input-5-c98b6c766be9>:77: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result.replace([np.inf, -np.inf], np.nan, inplace=True)


,COD,Sector,2016Q1,2016Q2,2016Q3,2016Q4,2016TOT,2017Q1,2017Q2,2017Q3,...,2022Q3,2022Q4,2022TOT,2023Q1,2023Q2,2023Q3,2023Q4,2023TOT,201624Q1,2024Q2
9,A,"Agricultura, ganadería, caza y silvicultura",0.160163,0.427247,0.176309,0.161887,0.226372,0.202844,0.370849,0.199829,...,2.008491,1.835347,2.368259,2.790311,4.939001,4.584225,6.572827,4.852999,9.651151,20.890247
10,B,Pesca,0.186062,0.376190,0.478360,0.302851,0.335229,0.241340,0.445806,0.485694,...,3.875453,2.144245,2.992823,2.714040,5.979462,8.881253,5.475174,5.760894,13.421658,28.517224
11,C,Explotación de minas y canteras,1.112255,1.119447,1.138542,1.178633,1.136903,1.356353,1.299227,1.367170,...,10.627272,14.770983,10.466802,17.114992,19.736109,27.772317,44.096455,27.239460,75.637819,79.886931
12,D,Industria manufacturera,0.208170,0.233490,0.236207,0.252380,0.232777,0.255670,0.301431,0.292486,...,3.064819,3.299245,2.679707,3.632909,5.362869,6.671398,8.990633,6.160451,13.393054,17.901616
13,E,"Electricidad, gas y agua",0.520372,0.582622,0.665306,0.549736,0.581623,0.691651,0.769012,0.971045,...,3.198987,2.706855,2.749301,4.763256,5.890734,8.061190,8.908115,7.037368,21.649533,35.915222
14,F,Construcción,0.101431,0.099779,0.109311,0.107979,0.104723,0.136186,0.138282,0.151461,...,0.942759,1.115825,0.908042,1.587619,1.681217,2.285326,2.875382,2.112343,4.480764,5.294576
15,G,"Comercio mayorista, minorista y reparaciones",0.131046,0.146785,0.153436,0.157482,0.147474,0.173806,0.187704,0.195861,...,1.766523,2.249865,1.657618,2.342240,3.115839,3.858526,5.575761,3.729057,8.229120,9.845707
16,H,Hoteles y restaurantes,0.122786,0.122030,0.138174,0.148963,0.133135,0.165149,0.175053,0.182611,...,1.108293,1.438221,1.125138,2.168652,2.320393,2.754394,3.899338,2.819381,7.023498,7.458428
17,I,"Transporte, almacenamiento y comunicaciones",0.147778,0.166264,0.165953,0.183382,0.165855,0.205705,0.233340,0.218852,...,1.531793,1.927408,1.503025,2.132391,2.644307,3.490919,4.447180,3.181090,7.556249,10.512287
18,J,Intermediación financiera,0.556724,0.527445,0.562360,0.661120,0.576785,0.727138,0.758724,0.723891,...,3.696288,3.875231,3.675195,4.913647,5.380369,4.752332,6.698196,5.424779,32.095269,39.454790


In [6]:
# Crear el DataFrame "merma_productividad" restando los valores entre los DataFrames
def calcular_merma_productividad(df_original, df_distraccion):
    # Separar las columnas no numéricas (COD y Sector)
    cod_sector = df_original.iloc[:, :2]  # Columnas 0 y 1
    # Seleccionar las columnas numéricas
    columnas_numericas = df_original.columns[2:]

    # Realizar la resta solo en las columnas numéricas
    df_original_numeric = df_original[columnas_numericas]
    df_distraccion_numeric = df_distraccion[columnas_numericas]

    # Calcular la merma restando los valores
    merma_numeric = df_original_numeric - df_distraccion_numeric

    # Unir las columnas no numéricas con el resultado de la resta
    merma = pd.concat([cod_sector, merma_numeric], axis=1)

    # Manejar valores NaN que puedan generarse en la operación
    merma.fillna(0, inplace=True)

    return merma

# Calcular el DataFrame "merma_productividad"
merma_productividad = calcular_merma_productividad(productividad_horas, productividad_horas_distraccion)

merma_productividad

,COD,Sector,2016Q1,2016Q2,2016Q3,2016Q4,2016TOT,2017Q1,2017Q2,2017Q3,...,2022Q3,2022Q4,2022TOT,2023Q1,2023Q2,2023Q3,2023Q4,2023TOT,201624Q1,2024Q2
9,A,"Agricultura, ganadería, caza y silvicultura",0.008898,0.023736,0.009795,0.008994,0.012576,0.011269,0.020603,0.011102,...,0.111583,0.101964,0.131570,0.155017,0.274389,0.254679,0.365157,0.269611,0.536175,1.160569
10,B,Pesca,0.010337,0.020899,0.026576,0.016825,0.018624,0.013408,0.024767,0.026983,...,0.215303,0.119125,0.166268,0.150780,0.332192,0.493403,0.304176,0.320050,0.745648,1.584290
11,C,Explotación de minas y canteras,0.061792,0.062191,0.063252,0.065480,0.063161,0.075353,0.072179,0.075954,...,0.590404,0.820610,0.581489,0.950833,1.096451,1.542906,2.449803,1.513303,4.202101,4.438163
12,D,Industria manufacturera,0.023130,0.025943,0.026245,0.028042,0.025864,0.028408,0.033492,0.032498,...,0.340535,0.366583,0.297745,0.403657,0.595874,0.741266,0.998959,0.684495,1.488117,1.989068
13,E,"Electricidad, gas y agua",0.057819,0.064736,0.073923,0.061082,0.064625,0.076850,0.085446,0.107894,...,0.355443,0.300762,0.305478,0.529251,0.654526,0.895688,0.989791,0.781930,2.405504,3.990580
14,F,Construcción,0.011270,0.011087,0.012146,0.011998,0.011636,0.015132,0.015365,0.016829,...,0.104751,0.123981,0.100894,0.176402,0.186802,0.253925,0.319487,0.234705,0.497863,0.588286
15,G,"Comercio mayorista, minorista y reparaciones",0.021841,0.024464,0.025573,0.026247,0.024579,0.028968,0.031284,0.032644,...,0.294420,0.374977,0.276270,0.390373,0.519307,0.643088,0.929294,0.621509,1.371520,1.640951
16,H,Hoteles y restaurantes,0.020464,0.020338,0.023029,0.024827,0.022189,0.027525,0.029175,0.030435,...,0.184715,0.239704,0.187523,0.361442,0.386732,0.459066,0.649890,0.469897,1.170583,1.243071
17,I,"Transporte, almacenamiento y comunicaciones",0.016420,0.018474,0.018439,0.020376,0.018428,0.022856,0.025927,0.024317,...,0.170199,0.214156,0.167003,0.236932,0.293812,0.387880,0.494131,0.353454,0.839583,1.168032
18,J,Intermediación financiera,0.092787,0.087907,0.093727,0.110187,0.096131,0.121190,0.126454,0.120649,...,0.616048,0.645872,0.612532,0.818941,0.896728,0.792055,1.116366,0.904130,5.349211,6.575798


In [7]:
# Crear el DataFrame "merma_productividad_valor" multiplicando los valores de "merma_productividad" por "df_horas"
def calcular_merma_productividad_valor(df_merma, df_horas):
    # Separar las columnas no numéricas (COD y Sector)
    cod_sector = df_merma.iloc[:, :2]  # Columnas 0 y 1
    # Seleccionar las columnas numéricas
    columnas_numericas = df_merma.columns[2:]

    # Realizar la multiplicación solo en las columnas numéricas
    df_merma_numeric = df_merma[columnas_numericas]
    df_horas_numeric = df_horas[columnas_numericas]

    # Calcular el valor de la merma multiplicando los valores
    merma_valor_numeric = df_merma_numeric * df_horas_numeric

    # Unir las columnas no numéricas con el resultado de la multiplicación
    merma_valor = pd.concat([cod_sector, merma_valor_numeric], axis=1)

    # Manejar valores NaN que puedan generarse en la operación
    merma_valor.fillna(0, inplace=True)

    return merma_valor

# Calcular el DataFrame "merma_productividad_valor"
merma_productividad_valor = calcular_merma_productividad_valor(merma_productividad, df_horas)

merma_productividad_valor

,COD,Sector,2016Q1,2016Q2,2016Q3,2016Q4,2016TOT,2017Q1,2017Q2,2017Q3,...,2022Q3,2022Q4,2022TOT,2023Q1,2023Q2,2023Q3,2023Q4,2023TOT,201624Q1,2024Q2
9,A,"Agricultura, ganadería, caza y silvicultura",16248.955684,45145.207222,21141.7369,21352.337223,25972.059257,21075.992135,40392.639421,23680.979227,...,244326.410446,245811.099517,277126.541352,292136.928872,528382.055139,561580.35114,916015.436071,574528.692806,1037096.306988,2272773.238451
10,B,Pesca,630.79281,1289.965726,1612.394168,1096.183157,1157.333965,898.485102,1718.198571,1912.174122,...,17241.210171,7528.35207,11649.244307,11072.560398,22465.532298,36247.725357,21580.757546,22841.6439,51662.404062,108429.14311
11,C,Explotación de minas y canteras,13313.445975,14355.565545,13975.691982,14087.378286,13933.020447,15776.033898,15383.336836,16572.466765,...,150455.938932,192118.298531,139085.134543,241778.82464,289122.785836,419931.916974,640607.033093,397860.140136,1047379.859293,1195670.875697
12,D,Industria manufacturera,96714.038096,109684.10266,116413.412058,121114.966355,110981.629792,116587.396727,136204.129717,144857.083355,...,1578263.776098,1666525.635951,1347393.304082,1863104.058529,2674667.34367,3401420.469734,4553811.298581,3123250.792629,6583542.008539,8501618.369019
13,E,"Electricidad, gas y agua",9809.387187,12868.120318,14747.321198,12454.049057,12469.71944,13737.110853,17691.124789,21811.576803,...,77073.171491,78869.277581,68096.101683,104248.874698,134568.078768,196677.186333,242783.892366,169569.508041,451147.206358,667542.234068
14,F,Construcción,29239.822263,29800.325436,34029.524272,33092.827724,31540.624924,40610.437862,41764.31495,51017.95271,...,344856.255886,378815.516841,304540.877572,531807.222758,595887.696915,871298.319896,989655.050745,747162.072578,1330240.813822,1626814.792557
15,G,"Comercio mayorista, minorista y reparaciones",134143.549458,155171.109151,169141.373589,172663.332275,157779.841118,179102.203582,200029.977095,228212.950807,...,2191355.426122,2538013.301298,1971066.64712,2838687.063988,3811130.022826,4944315.75099,6838454.877707,4608146.928878,10115167.253904,12352067.829095
16,H,Hoteles y restaurantes,22809.325353,21163.740891,24947.736701,27653.891575,24143.67363,31144.637955,29724.34584,34323.70734,...,219672.82196,276266.88702,211884.575692,432402.655903,405058.098745,579263.896617,832324.664303,562262.328892,1462065.827402,1341952.295888
17,I,"Transporte, almacenamiento y comunicaciones",38321.778089,45321.94771,44865.290281,47600.516304,44027.383096,54313.653193,61652.472435,60581.372482,...,396007.007637,475213.319956,385356.082945,558609.161909,709161.266582,878973.532225,1196599.516426,835835.869286,1993579.023592,2814137.592612
18,J,Intermediación financiera,40131.642437,43045.213297,47927.979598,52540.276928,45911.278065,55684.291889,57685.637993,60174.078865,...,301415.860199,308318.327842,288359.942716,358898.650825,453534.088659,418996.887558,527919.993538,439837.405145,2305216.668938,3176301.127897


In [8]:
# Crear el DataFrame "merma_productividad_porcentaje" dividiendo "merma_productividad_valor" sobre "df_vab_pb"
def calcular_merma_productividad_porcentaje(df_valor, df_vab):
    # Separar las columnas no numéricas (COD y Sector)
    cod_sector = df_valor.iloc[:, :2]  # Columnas 0 y 1
    # Seleccionar las columnas numéricas
    columnas_numericas = df_valor.columns[2:]

    # Realizar la división solo en las columnas numéricas
    df_valor_numeric = df_valor[columnas_numericas]
    df_vab_numeric = df_vab[columnas_numericas]

    # Calcular el porcentaje de merma productiva
    merma_porcentaje_numeric = df_valor_numeric.div(df_vab_numeric)

    # Unir las columnas no numéricas con el resultado de la división
    merma_porcentaje = pd.concat([cod_sector, merma_porcentaje_numeric], axis=1)

    # Manejar valores infinitos y nulos generados durante la división
    merma_porcentaje.replace([np.inf, -np.inf], np.nan, inplace=True)
    merma_porcentaje.fillna(0, inplace=True)

    return merma_porcentaje

# Calcular el DataFrame "merma_productividad_porcentaje"
merma_productividad_porcentaje = calcular_merma_productividad_porcentaje(merma_productividad_valor, df_vab_pb)

merma_productividad_porcentaje

<ipython-input-8-e47240fc7d35>:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merma_porcentaje.replace([np.inf, -np.inf], np.nan, inplace=True)


,COD,Sector,2016Q1,2016Q2,2016Q3,2016Q4,2016TOT,2017Q1,2017Q2,2017Q3,...,2022Q3,2022Q4,2022TOT,2023Q1,2023Q2,2023Q3,2023Q4,2023TOT,201624Q1,2024Q2
9,A,"Agricultura, ganadería, caza y silvicultura",0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,...,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632
10,B,Pesca,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,...,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632
11,C,Explotación de minas y canteras,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,...,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632,0.052632
12,D,Industria manufacturera,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
13,E,"Electricidad, gas y agua",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
14,F,Construcción,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
15,G,"Comercio mayorista, minorista y reparaciones",0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,...,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
16,H,Hoteles y restaurantes,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,...,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
17,I,"Transporte, almacenamiento y comunicaciones",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
18,J,Intermediación financiera,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,...,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857


In [9]:
# Guardar los DataFrames en un archivo Excel con múltiples pestañas
with pd.ExcelWriter('analisis_productividad.xlsx') as writer:
    df_vab_pb.to_excel(writer, sheet_name='vab_pb', index=False)
    df_puestos.to_excel(writer, sheet_name='puestos', index=False)
    df_horas.to_excel(writer, sheet_name='horas', index=False)
    productividad_puestos.to_excel(writer, sheet_name='productividad_puestos', index=False)
    productividad_horas.to_excel(writer, sheet_name='productividad_horas', index=False)
    productividad_horas_distraccion.to_excel(writer, sheet_name='productividad_horas_distraccion', index=False)
    merma_productividad.to_excel(writer, sheet_name='merma_productividad', index=False)
    merma_productividad_valor.to_excel(writer, sheet_name='merma_productividad_valor', index=False)
    merma_productividad_porcentaje.to_excel(writer, sheet_name='merma_productividad_porcentaje', index=False)